---
# Ensure that this title is the same as the one in `myst.yml`
title: Developer Notes - Core Team Template
abstract: |
  This tutorial describes how you visualize and extract kinetic parameters from a fluorescent plate reader experiment to enable consistent data reporting across the nucleus ecosystem. Within a few minutes, datasets can be rendered into a form suitable for sharing.
---

A shared understanding of how to interpret data from CFE systems would speed the development of measurements and standards towards improved reproducibility and address challenges in interpreting and comparing existing and future data, including data from failed experiments. Time-course measurements of the product expressed in the cytosol are favored over endpoint measurements, despite the time, labor, and costs involved, to obtain a more complete and informative view of a cytosol. This package helps to standardize the reporting of measurements as a reduced quantity, such as a mean value, with uncertainty and include a baseline from negative control measurements. The collection of such data enable reaction optimization and the development of predictive modeling tools.

:::{admonition} Step 0: experimenta setup
:class: attention

In order to make the most of this toolkit it is necessary to ensure that your has been designed following best practices to include replicates and controls. When designing an experiment we recommend following our platemap guide. Additionally, the following code makes some assumptions regarding the format of the incoming data. If you are using a Cytation instrument see our Getting started guide. Otherwise, it might be necessary to write some glue code to transform your raw data into a format compatible with this library. 
:::

:::{admonition} Step 1: python imports
:class: note

First we import standard python packages alongside our custom library called [cdk](https://github.com/bnext-bio/bnext/blob/cdk-layout/cdk/src/cdk/analysis/cytosol/platereader.py). Currently, this package contains several functions:

- `load_platereader_data()`
- `plot_plate()`
- `plot_curves_by_name()`
- `plot_curves()`
- `find_steady_state()`
- `plot_steady_state()`
- `kinetic_analysis()`
- `plot_kinetic_analysis()`

In what follows, we will demonstrate the use of these functions on some sample data.
:::

In [ ]:
%load_ext autoreload
%autoreload 2

from src.cdk.analysis.cytosol import platereader as pr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot.pandas
import src.cdk.logging

log = src.cdk.logging.setup_logging()
pr.plot_setup()

:::{admonition} step 2: load data
:class: note

Load your fluorescent timeseries data and experimental platemap. They should be formatted as .txt and .csv files, respectively. The `data` output will be used in subsequent plotting functions
:::

In [ ]:
data, platemap = pr.load_platereader_data("tests/test_data/cytation_dna_sweep.txt", "tests/test_data/platemap.csv")
data.head()

Verify the contents of the platemap by running `platemap`. Notice that this platemap contains two separate Experiments `Concentration` and `Artifact`. The Experiment parameter will be used in some of the downstream plotting functions below. See TODO: platemap guide for more information on setting up a platemap.

In [ ]:
platemap

:::{admonition} step 3: inspect data
:class: note

Use `plot_plate()` to look at plots of plate reader data, laid out the same as the plate.
:::

In [ ]:
pr.plot_plate(data);

:::{admonition} collect timeseries data
:class: seealso

Use `plot_curves_by_name()` to plot all curves, with one plot per Experiment.

NOTE: is the name of this function clear?
NOTE: should include a note - probably in guide for platemaps - that describes what an "Experiment" is
:::

In [ ]:
pr.plot_curves_by_name(data=data);

:::{admonition} collect timeseries data
:class: seealso

Use `plot_curves()` to plot curves with control over how the data is separated.

NOTE: is there a more interesting use case that we can show that highlights where this is different between the function above
:::

In [ ]:
pr.plot_curves(data, hue="Name", col="Experiment");

## Holoviz

In [ ]:
data.hvplot(
    by=['[DNA Template] (ng/uL)'],
    groupby=['Experiment'],
    kind='line',
    logy=True,
    x='Time',
    y=['Data'],
    legend='bottom_right',
    widget_location='top',
    height=500
)

data.hvplot.explorer(x="Time", y="Data", by=["Name"], groupby=["Experiment"])